In [1]:
# 참고 https://alex-blog.tistory.com/entry/%EB%8D%B0%EC%9D%B4%ED%84%B0-%EB%B6%84%EC%84%9D%EC%9D%84-%EC%9C%84%ED%95%9C-Python-%EA%B3%B5%EA%B3%B5%EB%8D%B0%EC%9D%B4%ED%84%B0-api-%ED%81%AC%EB%A1%A4%EB%A7%81

import pandas as pd
import json
import requests
from bs4 import BeautifulSoup

In [2]:
url = 'http://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList'

# stnIds = 강원 울산 경북
# 강원 stnIds = 90, 93, 95, 100, 102, 104, 105, 106, 114, 212, 216, 217, 121, 211
# 울산 stnIds = 152
# 경북 stnIds = 138, 283, 136, 279, 272, 281, 273, 276, 278, 276, 277, 271, 130

# 포항, 경주, 김천, 안동, 구미, 영주, 영천, 상주
# 문경 경산, 군위군, 의성군, 청송군, 영양군, 영덕군, 청도군, 고령군
# 성주군, 칠곡군, 예천군, 봉화군, 울진군, 울릉군

# 날씨 데이터가 없는 지역
# 김천, 경산, 군위군, 영양군, 청도군, 고령군, 성주군, 성주군, 칠곡군, 예천군, 봉화군, 울릉군


# b) 요청 메시지 명세
queryParams = '?' + \
              'ServiceKey=' + 'PNLH8BSwmhD8GsrOyOlmkIYv49lfU2CsjbGSdYTMUD9CIxl2MA%2F0JGU22moSzyr3EYpn7KFdA82qrjZSyYt6kQ%3D%3D' + \
              '&pageNo='+ '1' + \
              '&numOfRows='+ '999' + \
              '&dataType='+ 'JSON' + \
              '&dataCd='+ 'ASOS' + \
              '&dateCd='+ 'DAY' + \
              '&startDt='+ '20110101' + \
              '&endDt='+ '20211231'  + \
              '&stnIds='+ '108'# \        필요 지역 넣기

In [3]:
# 작동확인 1
url + queryParams # 확인

'http://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList?ServiceKey=PNLH8BSwmhD8GsrOyOlmkIYv49lfU2CsjbGSdYTMUD9CIxl2MA%2F0JGU22moSzyr3EYpn7KFdA82qrjZSyYt6kQ%3D%3D&pageNo=1&numOfRows=999&dataType=JSON&dataCd=ASOS&dateCd=DAY&startDt=20110101&endDt=20211231&stnIds=108'

In [4]:
region_list = [90, 93, 95, 100, 102, 104, 105, 106, 114, 212, 216, 217, 121, 211, 
               152, 138, 283, 136, 279, 272, 281, 273, 276, 278, 276, 277, 271, 130]

In [5]:
# 3분 40초 소요

wether_data = []

for i in region_list:
    queryParams = '?' + \
              'ServiceKey=' + 'PNLH8BSwmhD8GsrOyOlmkIYv49lfU2CsjbGSdYTMUD9CIxl2MA%2F0JGU22moSzyr3EYpn7KFdA82qrjZSyYt6kQ%3D%3D' + \
              '&pageNo='+ '1' + \
              '&numOfRows='+ '999' + \
              '&dataType='+ 'JSON' + \
              '&dataCd='+ 'ASOS' + \
              '&dateCd='+ 'DAY' + \
              '&startDt='+ '20120101' + \
              '&endDt='+ '20211231'  + \
              '&stnIds='+  str(i) # \        필요 지역 넣기
    
    result = requests.get(url + queryParams)
    js = json.loads(result.content)
    data = pd.DataFrame(js['response']['body']['items']['item'])
    wether_data.append(data)

In [6]:
# result = requests.get(url + queryParams)
# js = json.loads(result.content)
# data = pd.DataFrame(js['response']['body']['items']['item'])

In [7]:
data.tail()

,stnId,stnNm,tm,avgTa,minTa,minTaHrmt,maxTa,maxTaHrmt,mi10MaxRn,mi10MaxRnHrmt,...,avgM05Te,avgM10Te,avgM15Te,avgM30Te,avgM50Te,sumLrgEv,sumSmlEv,n99Rn,iscs,sumFogDur
994,130,울진,2014-09-21,19.0,13.3,0154,24.2,1025,,,...,,,,,,,,,,
995,130,울진,2014-09-22,19.3,14.1,0620,23.7,1307,,,...,,,,,,,,,,
996,130,울진,2014-09-23,19.9,14.5,0551,24.3,1607,0.2,,...,,,,,,,,,{비}2226-{비}{강도0}2400-,
997,130,울진,2014-09-24,17.4,16.5,1016,18.4,0001,4.7,1042,...,,,,,,,,,-{비}-{비}{강도1}0300-{비}{강도1}0600-{비}{강도1}0900-{비...,
998,130,울진,2014-09-25,19.1,16.5,2313,23.1,1050,0.2,,...,,,,,,,,,{비}0215-0255. {비}0425-0457.,


In [10]:
# csv로 내보내기

for i in range(len(wether_data)):
    wether_dataframe = wether_data[i]
    wether_dataframe_name = wether_dataframe.loc[0, "stnNm"]
    # index = str(region_list[i])
    wether_dataframe.to_csv('./wether_data/wether_data ' + wether_dataframe_name + '.csv', encoding= 'euc-kr')